In [2]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
dataset = load_dataset("conll2003")
train_data = dataset["train"]
validation_data = dataset["validation"]
test_data = dataset["test"]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
sample = train_data[0]
print(sample)

{'id': '0', 'tokens': ['EU', 'rejects', 'German', 'call', 'to', 'boycott', 'British', 'lamb', '.'], 'pos_tags': [22, 42, 16, 21, 35, 37, 16, 21, 7], 'chunk_tags': [11, 21, 11, 12, 21, 22, 11, 12, 0], 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}


In [5]:
len(train_data)

14041

In [6]:
import torch
import torch.nn as nn
from transformers import BertTokenizer, BertModel
from datasets import load_dataset

In [7]:
import gensim.downloader as api

In [8]:
import numpy as np

In [9]:
w2v = api.load('word2vec-google-news-300')

[==================================================] 100.0% 1662.8/1662.8MB downloaded


In [12]:
class LSTM_MoE(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_experts):
        super(LSTM_MoE, self).__init__()
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True).float()
        self.moe = MoE(hidden_dim, hidden_dim, num_experts)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_dim, num_labels)

    def forward(self, x):
        out, _ = self.lstm1(x)
        out, _ = self.moe(out)
        out, _ = self.lstm2(out)
        out = self.fc(out)
        return out


In [13]:
class MoE(nn.Module):
    def __init__(self, input_dim, output_dim, num_experts):
        super(MoE, self).__init__()
        self.num_experts = num_experts
        self.experts = nn.ModuleList([nn.Linear(input_dim, 128) for _ in range(num_experts)])
        self.gating_network = nn.Linear(input_dim, num_experts)

    def forward(self, x):
        gates = torch.sigmoid(self.gating_network(x))
        expert_outputs = [expert(x) for expert in self.experts]
        expert_outputs_tensor = torch.stack(expert_outputs, dim=0)
        # for gate, expert_output in zip(gates, expert_outputs):
        #   print(gate.shape)
        #   print(expert_output.shape)
        expert_outputs_stacked = torch.stack(expert_outputs, dim=1)
        moe_output = torch.sum(gates.unsqueeze(2) * expert_outputs_stacked, dim=1)

        return moe_output, gates


In [17]:
input_dim = 300
hidden_dim = 128
num_layers = 2
num_experts = 16
num_labels = 10
model = LSTM_MoE(input_dim, hidden_dim, num_layers, num_experts)

In [18]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 5

for epoch in range(num_epochs):
    loss = 0
    for batch in train_data:
        optimizer.zero_grad()
        tokens = batch['tokens']
        token_embeddings = []
        for token in tokens:
            if token in w2v:
                embedding = w2v[token]
                token_embeddings.append(embedding)
            else:
                token_embeddings.append(np.zeros(w2v.vector_size))

        tokens_array = np.array(token_embeddings)
        inputs = torch.tensor(tokens_array, dtype=torch.float)
        # for input in inputs:
        #   print(input.shape)

        labels = torch.tensor(batch['ner_tags'])
        # print("input :", inputs.shape)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss:.4f}%")

Epoch: 0, Loss: 0.0980%
Epoch: 1, Loss: 0.0349%
Epoch: 2, Loss: 0.0437%
Epoch: 3, Loss: 0.0102%
Epoch: 4, Loss: 0.0096%


In [19]:
model.eval()

total_loss = 0.0
total_correct = 0
total_samples = 0

for batch in test_data:
    inputs = batch['tokens']
    labels = torch.tensor(batch['ner_tags'])

    token_embeddings = []
    for token in inputs:
        if token in w2v:
            embedding = w2v[token]
            token_embeddings.append(embedding)
        else:
            token_embeddings.append(np.zeros(w2v.vector_size))

    tokens_array = np.array(token_embeddings)
    inputs = torch.tensor(tokens_array, dtype=torch.float)

    with torch.no_grad():
        outputs = model(inputs)

    loss = criterion(outputs, labels)
    total_loss += loss.item()

    _, predicted = torch.max(outputs, 1)
    total_correct += (predicted == labels).sum().item()
    total_samples += labels.size(0)

average_loss = total_loss / len(test_data)
accuracy = total_correct / total_samples

print(f"Average Loss: {average_loss:.4f}, Accuracy: {accuracy * 100:.2f}%")

Average Loss: 0.2923, Accuracy: 95.47%
